# Multi-objective Robust Optimization (MORO)


This exercise demostrates the application of MORO on the lake model. In contrast to the exercises in previous weeks, we will be using a slightly more sophisticated version of the problem. For details see the MORDM assignment for this week.

## Setup MORO

Many objective robust optimization aims at finding decisions that are robust with respect to the various deeply uncertain factors. For this, MORO evalues each candidate decision over a set of scenarios. For each outcome of interest, the robusntess over this set is calculated. A MOEA is used to maximize the robustness. 

For this assignment, we will be using a domain criterion as our robustness metric. The table below lists the rules that you should use for each outcome of interest.

|Outcome of interest| threhsold  |
|-------------------|------------|
| Maximum pollution | $\leq$ 0.75|
| Inertia           | $\geq$ 0.6 |
| Reliability       | $\geq$ 0.99|   
| Utility           | $\geq$ 0.75|

**1) Implement a function for each outcome that takes a numpy array with results for the outcome of interest, and returns the robustness score**

In [3]:
import math
import numpy as np

from scipy.optimize import brentq

from ema_workbench import (Model, RealParameter, ScalarOutcome, Constant,
                           ema_logging, MultiprocessingEvaluator)
from ema_workbench.em_framework.samplers import sample_uncertainties
ema_logging.log_to_stderr(ema_logging.INFO)
from dps_lake_model import lake_model as lake_problem

# instantiate the model
lake_model = Model('lakeproblem', function=lake_problem)
# specify uncertainties
lake_model.uncertainties = [RealParameter('b', 0.1, 0.45),
                            RealParameter('q', 2.0, 4.5),
                            RealParameter('mean', 0.01, 0.05),
                            RealParameter('stdev', 0.001, 0.005),
                            RealParameter('delta', 0.93, 0.99)]

# set levers
lake_model.levers = [RealParameter("c1", -2, 2),
                        RealParameter("c2", -2, 2),
                        RealParameter("r1", 0, 2),
                        RealParameter("r2", 0, 2),
                        RealParameter("w1", 0, 1)
                        ]

# specify outcomes
lake_model.outcomes = [ScalarOutcome('max_P'),
                        ScalarOutcome('utility'),
                        ScalarOutcome('inertia'),
                        ScalarOutcome('reliability')]

# override some of the defaults of the model
lake_model.constants = [Constant('alpha', 0.41),
                        Constant('nsamples', 100),
                        Constant('myears', 100)]

# setup and execute the robust optimization
def signal_to_noise(data):
    mean = np.mean(data)
    std = np.std(data)
    sn = mean / std
    return sn

MAXIMIZE = ScalarOutcome.MAXIMIZE  # @UndefinedVariable
MINIMIZE = ScalarOutcome.MINIMIZE  # @UndefinedVariable
robustnes_functions = [
    ScalarOutcome(
        'mean p',
        kind=MINIMIZE,
        variable_name='max_P',
        function=np.mean),
    ScalarOutcome(
        'std p',
        kind=MINIMIZE,
        variable_name='max_P',
        function=np.std),
    ScalarOutcome(
        'sn reliability',
        kind=MAXIMIZE,
        variable_name='reliability',
        function=signal_to_noise)]
n_scenarios = 10
scenarios = sample_uncertainties(lake_model, n_scenarios)
nfe = 1000

with MultiprocessingEvaluator(lake_model) as evaluator:
    evaluator.robust_optimize(robustnes_functions, scenarios, nfe=nfe,
                                epsilons=[0.1, ] * len(robustnes_functions),
                                population_size=5)

[MainProcess/INFO] pool started
[MainProcess/INFO] generation 0: 0/1000 nfe
[MainProcess/INFO] generation 5: 29/1000 nfe
[MainProcess/INFO] generation 10: 58/1000 nfe
[MainProcess/INFO] generation 15: 88/1000 nfe
[MainProcess/INFO] generation 20: 118/1000 nfe
[MainProcess/INFO] generation 25: 148/1000 nfe


KeyboardInterrupt: 

In [1]:
from ema_workbench import (RealParameter, ScalarOutcome, Constant,
                           Model)
import functools

def robustness(direction, threshold, data):
    if direction == SMALLER:
        return np.sum(data<=threshold)/data.shape[0]
    else:
        return np.sum(data>=threshold)/data.shape[0]

def maxp(data):
    return np.sum(data<=0.75)/data.shape[0]
    
SMALLER = 'SMALLER'
LARGER = 'LARGER'

maxp2 = functools.partial(robustness, SMALLER, 0.75)
inertia = functools.partial(robustness, LARGER, 0.6)
reliability = functools.partial(robustness, LARGER, 0.99)
utility = functools.partial(robustness, LARGER, 0.75)

MAXIMIZE = ScalarOutcome.MAXIMIZE
MINIMIZE = ScalarOutcome.MINIMIZE
# choose one, not sure which. 
# First is okay but not advised. 
# robustnes_functions = [ScalarOutcome('90th percentile max_p', kind=MINIMIZE,
#                              variable_name='max_P', function=maxp),
#                        ScalarOutcome('10th percentile reliability', kind=MAXIMIZE,
#                              variable_name='reliability', function=reliability),
#                        ScalarOutcome('10th percentile inertia', kind=MAXIMIZE,
#                              variable_name='inertia', function=inertia),
#                        ScalarOutcome('10th percentile utility', kind=MAXIMIZE,
#                              variable_name='utility', function=utility)]

robustnes_maxp_function = [ScalarOutcome('max_p', kind=MINIMIZE,
                             variable_name='max_P', function=maxp)]
robustnes_reliability_function = [ScalarOutcome('reliability', kind=MAXIMIZE,
                             variable_name='reliability', function=reliability)]
robustnes_inertia_function = [ScalarOutcome('inertia', kind=MAXIMIZE,
                             variable_name='inertia', function=inertia)]
robustnes_utility_function = [ScalarOutcome('utility', kind=MAXIMIZE,
                             variable_name='utility', function=utility)]




**2) Generate 4 random release policies, and evaluate them over 500 scenarios. Sample the scenarios using Monte Carlo sampling. Next evaulate your robustness function for 1, 2, 3, ... 500 scenarios for each outcome and visualize this. What can you tell about the convergernce of the robusntess metric as a function of the number of scenarios?**

In [2]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments)
from ema_workbench.em_framework.evaluators import MC
from dps_lake_model import lake_model as lakeModel
lake_model = Model('lakeproblem', function=lakeModel)
# instantiate model
lake_model.uncertainties = [RealParameter('b', 0.1, 0.45),
                                RealParameter('q', 2.0, 4.5),
                                RealParameter('mean', 0.01, 0.05),
                                RealParameter('stdev', 0.001, 0.005),
                                RealParameter('delta', 0.93, 0.99)]

# set levers
lake_model.levers = [RealParameter("c1", -2, 2),
                         RealParameter("c2", -2, 2),
                         RealParameter("r1", 0, 2),
                         RealParameter("r2", 0, 2),
                         RealParameter("w1", 0, 1)
                         ]

# specify outcomes
lake_model.outcomes = [ScalarOutcome('max_P'),
                           ScalarOutcome('utility'),
                           ScalarOutcome('inertia'),
                           ScalarOutcome('reliability')]

# override some of the defaults of the model
lake_model.constants = [Constant('alpha', 0.41),
                            Constant('nsamples', 100),
                            Constant('myears', 100)]


In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(lake_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=500, policies=4, uncertainty_sampling=MC)

In [7]:
experiments, outcomes = results
# print(experiments)
maxp_exp = experiments[experiments['policy']%4==0]
maxp_exp.drop(columns=['scenario', 'policy', 'model'], inplace=True)
# print(maxp_exp)
experiments, outcomes = results
utility_exp = experiments[experiments['policy']==1]
experiments, outcomes = results
reliability_exp = experiments[experiments['policy']==2]
experiments, outcomes = results
inertia_exp = experiments[experiments['policy']==3]

# maxp_out = outcomes['max_P'][0:499]
selected_scenarios = []

from ema_workbench import Scenario

for index, row in maxp_exp.iterrows():
        selected_scenarios.append(Scenario(str(index), ** row.to_dict()))

So i think this stuff below was not supposed to be done in q2, although i am not completely sure. 

I also think it does work, but it is super slow to run this for 1000 nfe and 500 scenarios. So i might have been a bit impatient when running this. However in my defense, when you wait for 15 minutes and nothing happens it usually means you are having a deadlock, usually. So let me know what you think should happen here, i actually do not know. 

In [16]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments)
from ema_workbench.em_framework.evaluators import MC
from ema_workbench.em_framework.samplers import sample_uncertainties
from dps_lake_model import lake_model as lakeModel
import pandas as pd
import numpy as np

n_scenarios = 500
scenarios = sample_uncertainties(lake_model, n_samples=500)
nfe = 1000

with MultiprocessingEvaluator(lake_model) as evaluator:
    evaluator.robust_optimize(robustnes_maxp_function, scenarios=scenarios, nfe=nfe,
                                  epsilons=[0.1], logging_freq=1)

[MainProcess/INFO] generation 0: 0/1000 nfe


In [8]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments)
from ema_workbench.em_framework.evaluators import MC
from ema_workbench.em_framework.samplers import sample_uncertainties
from dps_lake_model import lake_model as lakeModel

n_scenarios = 500
# scenarios = sample_uncertainties(lake_model, n_scenarios, sampler=MC)
nfe=1000

ema_logging.log_to_stderr(ema_logging.INFO)
with MultiprocessingEvaluator(lake_model) as evaluator:
    results = evaluator.robust_optimize(robustnes_maxp_function, reference=selected_scenarios, nfe=nfe,epsilons=[0.1], logging_freq=1)


[MainProcess/INFO] generation 0: 0/1000 nfe


## Searching for candidate solutions
Set up the robust optimization problem using the robustness functions you have specified. Assume that you will need 50 scenarios for estimating the robustness. Use $\epsilon$-progress and hypervolume to track convergence. Solve the optimization problem. As $\epsilon$ values, you can assume 0.05 for each of the four robustness metrics.

*note: this optimization problem is computationally very expensive. Develop and test your code using a sequential evaluator, a low number of function evaluations (e.g., 200), and a low number of scenarios (e.g., 5). Once everything seems to be working replace the sequential evaluator with an multiprocessing or ipyparallel evaluator, and increase the number of nfe and scenarios*.


In [6]:
from ema_workbench import (MultiprocessingEvaluator, SequentialEvaluator, ema_logging,
                           perform_experiments)
from ema_workbench.em_framework.evaluators import MC
from ema_workbench.em_framework.samplers import sample_uncertainties
from dps_lake_model import lake_model as lakeModel
import pandas as pd
import numpy as np

# robustnes_maxp_function 
# robustnes_reliability_function 
# robustnes_inertia_function 
# robustnes_utility_function 
# lake_model

nfe = 200
n_scen = 5
scenario = sample_uncertainties(lake_model, n_scen)

ema_logging.log_to_stderr(ema_logging.INFO)
with MultiprocessingEvaluator(lake_model) as evaluator:
    maxp_res = evaluator.robust_optimize(robustnes_maxp_function, scenarios=scenario, nfe=nfe,epsilons=[0.05])

ema_logging.log_to_stderr(ema_logging.INFO)
with MultiprocessingEvaluator(lake_model) as evaluator:
    reliability_res = evaluator.robust_optimize(robustnes_reliability_function, scenarios=scenario, nfe=nfe,epsilons=[0.05])

ema_logging.log_to_stderr(ema_logging.INFO)
with MultiprocessingEvaluator(lake_model) as evaluator:
    utility_res = evaluator.robust_optimize(robustnes_utility_function, scenarios=scenario, nfe=nfe,epsilons=[0.05])

ema_logging.log_to_stderr(ema_logging.INFO)
with MultiprocessingEvaluator(lake_model) as evaluator:
    inertia_res = evaluator.robust_optimize(robustnes_inertia_function, scenarios=scenario, nfe=nfe,epsilons=[0.05])


[MainProcess/INFO] pool started
[MainProcess/INFO] generation 0: 0/200 nfe
[MainProcess/INFO] optimization completed, found 1 solutions
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] generation 0: 0/200 nfe
[MainProcess/INFO] optimization completed, found 1 solutions
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] generation 0: 0/200 nfe
[MainProcess/INFO] optimization completed, found 1 solutions
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] generation 0: 0/200 nfe
[MainProcess/INFO] optimization completed, found 1 solutions
[MainProcess/INFO] terminating pool


**Plot your $\epsilon$-progress to evaluate convergergence, and visualize the trade-offs using parallel coordinate plots**

**What does this plot tell us about the tradeoffs and conflicting objectives?**

## Re-evaluate candidate solutions under uncertainty

We have used only 50 scenarios for the optimization. Take the results and re-evaluate them over a larger set (assume 1000 scenarios). How different are your results? What does this imply for the assumption of 50 scenarios during robust optimization.

*hint: use the to_dict method on a dataframe, next generate Policy objects in a list expression by iterating over the dicts returned by the to_dict method*

## Comparison
If you have time, import your solutions found for MORDM and re-evaluate them over the same set of scnearios as used for re-evaluating the MORO results. Compare the robustness of MORDM and MORO, what do you observe?